In [180]:
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.io
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from scipy.signal import welch
%matplotlib inline


In [181]:
#fft_result = np.fft.fft(EEG2)
#freq = np.fft.fftfreq(len(EEG2), d=1/Fs)
#power_spectrum = np.abs(fft_result)
#plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
#plt.title("EEG Power Spectrum")
#plt.xlabel("Frequency (Hz)")
#plt.ylabel("Power")
#plt.yscale('log')
#plt.show()

In [182]:
path= "/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/"

import os

def fast_scandir(dirname):
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return sorted(subfolders)

patients_path = fast_scandir(path)
patients_path

['/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0284',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0286',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0296',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0299',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0303',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0306',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0311',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0312',
 '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training

In [183]:


def get_eeg_files(directory):
    eeg_files = []

    for root, _, files in os.walk(directory):

        for file in files:

            if file.endswith("EEG.mat"):

                absolute_path = os.path.abspath(os.path.join(root, file))
                eeg_files.append(absolute_path)
    if len(eeg_files) == 0:
        print('No files found.')
        return None

    print(f'Found {len(eeg_files)} files.')
    print(f"Patient's number: {directory[-4:]}")
    return sorted(eeg_files)




EEG_file_list = get_eeg_files(patients_path[6])


Found 6 files.
Patient's number: 0311


In [184]:

hours =[]
EEG_list = []
scaler = StandardScaler()

for file_path in EEG_file_list[1:-1]:

    eeg = scipy.io.loadmat(file_path)
    eeg = eeg['val']
    eeg = eeg.astype(float)
    #for line in eeg:
        #temp_line = line.reshape(1, -1)
        #temp_line = scaler.fit_transform(temp_line)
        #line = temp_line.reshape(-1)
    #eeg = np.mean(eeg,axis=0)
    EEG_list.append(eeg)

    hour = file_path[-11:-8]
    hours.append(hour)

len(hours)

KeyboardInterrupt: 

In [ ]:
EEG_list_ar = np.array(EEG_list)

EEG_list_ar.shape

(4, 20, 7372800)

In [ ]:
def reduce_EEG(EEG, rate_of_reduction = 5, original_freq = 500):
    rate_of_reduction = np.round(rate_of_reduction)
    if len(EEG) % rate_of_reduction == 0:
        print(f'EEG reduced to a {original_freq/rate_of_reduction} Hz frequence.')
        return EEG.reshape(-1, int(rate_of_reduction) ).mean(axis=1)
    else:
        print('Data will be cut due to undivisible length.')
        remaining = int(len(EEG) % rate_of_reduction)
        print(f'Points lost: {remaining}')
        EEG = EEG[:-remaining]
        print(f'EEG reduced to a {original_freq/rate_of_reduction} Hz frequence.')
        return EEG.reshape(-1, int(rate_of_reduction) ).mean(axis=1)

In [ ]:
original_freq = 500
new_freq = 100
psds = []
for eeg in EEG_list:
    eeg = reduce_EEG(eeg,rate_of_reduction=original_freq/new_freq,original_freq=original_freq)
    f, temp_psd = welch(eeg, fs=new_freq, nperseg=4*new_freq)
    psds.append(temp_psd)


EEG reduced to a 100.0 Hz frequence.
EEG reduced to a 100.0 Hz frequence.


EEG reduced to a 100.0 Hz frequence.
EEG reduced to a 100.0 Hz frequence.
EEG reduced to a 100.0 Hz frequence.


In [ ]:
fig = go.Figure()

index = 0
for psd in psds:
    color_intensity = min((1 - index/len(psds) ),1) # This is to change the color of each line. Plotly doesn't like to do it, so I had to force it with an f string and rgb values.
    fig.add_trace(go.Scatter(x = f, y = psd,
        mode='lines',

        line=dict( color= f"rgb({255*color_intensity},0,{255-(255*color_intensity)})"),

        opacity = 0.5,

    ))
    index += 1

fig.update_yaxes(type='log')
fig.show()
print(color_intensity)

0.19999999999999996


In [187]:
psds = np.array(psds)
psds[1,:]

array([ 2503.91417737, 18945.94674072, 14141.83479442, 11159.06294773,
       10478.7702675 , 17676.4325664 , 11480.41860214,  5406.00518554,
        4502.69338264,  3554.71827873,  2431.20200648,  2250.31445973,
        2784.80699225,  3182.17780546,  3209.67948224,  3147.55067934,
        3024.45558658,  2691.83086823,  2378.0820974 ,  2321.02776377,
        2253.62611594,  2138.39853887,  2099.91472522,  2190.57425983,
        2261.28765535,  2334.11412629,  2300.55796416,  2163.63880896,
        2012.99689301,  1842.228215  ,  1754.53011389,  1720.50841182,
        1673.14651114,  1661.77739571,  1637.74382494,  1516.08754286,
        1494.34942963,  1484.14120886,  1440.61133629,  1405.9479985 ,
        1345.82390842,  1355.94184643,  1411.68859505,  1398.92042917,
        1345.95701268,  1373.42015439,  1366.37550355,  1382.12762172,
        2651.42206773,  4465.63658402,  1736.51312834,  1271.17721451,
        1289.71500621,  1302.06658236,  1284.60596771,  1218.83218656,
      

In [162]:
hours

['046', '047', '048', '049', '050']

In [163]:
hours2 = pd.DataFrame(hours,columns=['hours'])
psds_df = pd.DataFrame(psds)
psds_df = pd.concat([psds_df, hours2], axis=1)
psds_df = psds_df.groupby(by='hours',as_index=True).mean()
psds_df = psds_df.div(psds_df[0],axis=0)

In [164]:
psds_df[0]

hours
046    1.0
047    1.0
048    1.0
049    1.0
050    1.0
Name: 0, dtype: float64

In [165]:
# ini_psds = psds[:,:25]

# ini_psds = ini_psds.T/(ini_psds[:,0])
# ini_psds.shape

In [168]:
px.line(psds_df,log_y=False,)

In [ ]:
ini_psds.T.shape

In [ ]:
EEG_file_list[50]